<a href="https://colab.research.google.com/github/humanet1603/krx_using_api/blob/main/pykrx_ncsoft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pykrx

In [ ]:

from pykrx import stock
import pandas as pd
from datetime import datetime, timedelta

def fetch_stock_data(ticker, years):
    """Fetch stock data for a given ticker and number of years."""
    end_date = datetime.now()
    start_date = end_date - timedelta(days=years * 365)
    # Format dates for the pykrx API
    fmt_start_date = start_date.strftime('%Y%m%d')
    fmt_end_date = end_date.strftime('%Y%m%d')

    # Fetch OHLCV data using pykrx
    df = stock.get_market_ohlcv_by_date(fmt_start_date, fmt_end_date, ticker)
    return df

def save_data_to_csv(data, filename):
    """Save pandas DataFrame to a CSV file."""
    data.to_csv(filename)
    print(f"Data saved to {filename}")

# Example usage
ticker = "036570"  # 엔씨소프트의 티커 코드
data = fetch_stock_data(ticker, 22)  # 10년치 데이터
save_data_to_csv(data, "/content/SK_Hynix_22_years_OHLCV.csv")


In [ ]:
import pandas as pd
import numpy as np
from pykrx import stock
from datetime import datetime

def fetch_data(ticker, start, end):
    # 데이터를 가져오는 함수
    df = stock.get_market_ohlcv_by_date(start, end, ticker)
    return df

def calculate_macd(data, slow=26, fast=12, signal=9):
    # MACD 값을 계산하는 함수
    data['EMA_Fast'] = data['종가'].ewm(span=fast, adjust=False).mean()
    data['EMA_Slow'] = data['종가'].ewm(span=slow, adjust=False).mean()
    data['MACD'] = data['EMA_Fast'] - data['EMA_Slow']
    data['Signal'] = data['MACD'].ewm(span=signal, adjust=False).mean()
    return data

def generate_signals(data):
    # 거래 신호를 생성하는 함수
    data['Buy_Signal'] = (data['MACD'] > data['Signal']) & (data['MACD'].shift(1) <= data['Signal'].shift(1))
    data['Sell_Signal'] = (data['MACD'] < data['Signal']) & (data['MACD'].shift(1) >= data['Signal'].shift(1))
    return data

def backtest(data):
    # 백테스팅을 수행하고 결과를 저장하는 함수
    data['Position'] = 0
    data.loc[data['Buy_Signal'], 'Position'] = 1
    data.loc[data['Sell_Signal'], 'Position'] = 0
    data['Position'] = data['Position'].ffill().fillna(0)
    data['Market_Returns'] = data['종가'].pct_change()
    data['Strategy_Returns'] = data['Market_Returns'] * data['Position']
    data['Cumulative_Strategy_Returns'] = (1 + data['Strategy_Returns']).cumprod() - 1
    return data

def save_to_csv(data, filename):
    data.to_csv(filename)

# 사용 예
ticker = "000660"  # SK하이닉스
start_date = '20230101'
end_date = '20240430'
data = fetch_data(ticker, start_date, end_date)
data = calculate_macd(data)
data = generate_signals(data)
result = backtest(data)

# 결과 저장
save_to_csv(result[['종가', 'MACD', 'Signal', 'Buy_Signal', 'Sell_Signal', 'Cumulative_Strategy_Returns']], 'MACD_Backtest_Results.csv')


In [ ]:
import pandas as pd
import numpy as np
from pykrx import stock
from datetime import datetime

def fetch_data(ticker, start, end):
    # 데이터를 가져오는 함수
    df = stock.get_market_ohlcv_by_date(start, end, ticker)
    return df

def calculate_macd(data, slow=26, fast=12, signal=9):
    # MACD 값을 계산하는 함수
    data['EMA_Fast'] = data['종가'].ewm(span=fast, adjust=False).mean()
    data['EMA_Slow'] = data['종가'].ewm(span=slow, adjust=False).mean()
    data['MACD'] = data['EMA_Fast'] - data['EMA_Slow']
    data['Signal'] = data['MACD'].ewm(span=signal, adjust=False).mean()
    return data

def generate_signals(data):
    # 거래 신호를 생성하는 함수
    data['Buy_Signal'] = (data['MACD'] > data['Signal']) & (data['MACD'].shift(1) <= data['Signal'].shift(1))
    data['Sell_Signal'] = (data['MACD'] < data['Signal']) & (data['MACD'].shift(1) >= data['Signal'].shift(1))
    return data

def backtest(data):
    # 백테스팅을 수행하고 결과를 저장하는 함수
    data['Position'] = 0
    data.loc[data['Buy_Signal'], 'Position'] = 1
    data.loc[data['Sell_Signal'], 'Position'] = 0
    data['Position'] = data['Position'].ffill().fillna(0)
    data['Market_Returns'] = data['종가'].pct_change()
    data['Strategy_Returns'] = data['Market_Returns'] * data['Position']
    data['Cumulative_Strategy_Returns'] = (1 + data['Strategy_Returns']).cumprod() - 1
    return data

def save_to_csv(data, filename):
    data.to_csv(filename)

def save_signals_to_csv(data, filename):
    # 신호가 True인 데이터만 필터링하여 저장하는 함수
    signals = data[(data['Buy_Signal'] | data['Sell_Signal'])]
    signals.to_csv(filename, index=False)
    print(f"Signal data saved to {filename}")

# 사용 예
ticker = "000660"  # SK하이닉스
start_date = '20230101'
end_date = '20240430'
data = fetch_data(ticker, start_date, end_date)
data = calculate_macd(data)
data = generate_signals(data)
result = backtest(data)

# 결과 저장
save_to_csv(result[['종가', 'MACD', 'Signal', 'Buy_Signal', 'Sell_Signal', 'Cumulative_Strategy_Returns']], 'MACD_Backtest_Results.csv')

# 신호 데이터만 저장
save_signals_to_csv(result, 'MACD_Trading_Signals.csv')


In [ ]:
import pandas as pd
import numpy as np
from pykrx import stock
from datetime import datetime

def fetch_data(ticker, start, end):
    df = stock.get_market_ohlcv_by_date(start, end, ticker)
    return df

def calculate_macd(data, slow=26, fast=12, signal=9):
    data['EMA_Fast'] = data['종가'].ewm(span=fast, adjust=False).mean()
    data['EMA_Slow'] = data['종가'].ewm(span=slow, adjust=False).mean()
    data['MACD'] = data['EMA_Fast'] - data['EMA_Slow']
    data['Signal'] = data['MACD'].ewm(span=signal, adjust=False).mean()
    return data

def generate_signals(data):
    data['Buy_Signal'] = (data['MACD'] > data['Signal']) & (data['MACD'].shift(1) <= data['Signal'].shift(1))
    data['Sell_Signal'] = (data['MACD'] < data['Signal']) & (data['MACD'].shift(1) >= data['Signal'].shift(1))
    return data

def backtest(data):
    data['Position'] = 0
    data.loc[data['Buy_Signal'], 'Position'] = 1
    data.loc[data['Sell_Signal'], 'Position'] = 0
    data['Position'] = data['Position'].ffill().fillna(0)
    data['Market_Returns'] = data['종가'].pct_change()
    data['Strategy_Returns'] = data['Market_Returns'] * data['Position']
    data['Cumulative_Strategy_Returns'] = (1 + data['Strategy_Returns']).cumprod() - 1
    return data

def save_to_csv(data, filename):
    data.to_csv(filename)

def save_signals_to_csv(data, filename):
    signals = data[(data['Buy_Signal'] | data['Sell_Signal'])]
    signals.to_csv(filename)
    print(f"Trading signal data saved to {filename}")

# Use Example
ticker = "000660"  # SK하이닉스
start_date = '20230101'
end_date = '20240430'
data = fetch_data(ticker, start_date, end_date)
data = calculate_macd(data)
data = generate_signals(data)
result = backtest(data)

# Save overall results and signals
save_to_csv(result, 'MACD_Backtest_Results.csv')
save_signals_to_csv(result, 'MACD_Trading_Signals.csv')
